# **BASE ECONOMÍA**

En este notebook se realizará la limpieza de tablas descargadas de la página del banco central, entre la información que se tiene encontramos consumo, tasa de desempleo, e indicadores financieros (cuentas bancarias, saldos, etc.).

In [1]:
#========================================================#
#== Definimos las librerias necesarias para esta parte ==#
#========================================================#

import pandas as pd
import seaborn as sns
import pyarrow.parquet as pq
import pyarrow as pa
import os
import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
directorio = '/content/drive/MyDrive/CD-FINAL/datos de economia/Datos_crudos_1'
#directorio = 'Datos_crudos_1'

In [36]:
codigo_regiones = {
    'Libertador': 6,
    'Nuble': 16,
    'Maule': 7,
    'Metropolitana': 13,
    'Biobio': 8,
    'Aysén': 11,
    'Valparaiso': 5,
    'Arica': 15,
    'Loslagos': 10,
    'Araucania': 9,
    'Antofagasta': 2,
    'Tarapaca': 1,
    'Losrios': 14,
    'Coquimbo': 4,
    'Magallanes': 12,
    'Atacama': 3
}


## Importación y limpieza de tablas.



---


### TABLA1: Consumo de hogares e IPSFL

* Consumo de hogares: Es el gasto total realizado por las familias en bienes y
servicios para satisfacer sus necesidades. Incluye cosas como alimentación, vivienda, transporte, educación, salud, etc.
* IPSFL (Instituciones Privadas Sin Fines de Lucro): Son organizaciones no gubernamentales que brindan servicios a los hogares, como fundaciones, asociaciones, y ONGs. Su consumo está relacionado con los servicios que ofrecen (financiados por donaciones, cuotas, etc.).

En conjunto, este indicador mide cuánto se gasta en la región tanto por los hogares como por estas instituciones.

In [5]:
archivos_csv = glob.glob(os.path.join(directorio, "*.xlsx"))

# Almancenaré en un diccionario los dataframe
df_consumo = pd.DataFrame()

# Leer cada archivo CSV y almacenarlo en el diccionario
for i in archivos_csv:
    nombre_archivo = os.path.basename(i).split('.')[0]  # Nombre del archivo sin la extensión
    if 'Consumo_' in nombre_archivo:
      tablas = pd.read_excel(i, engine='openpyxl')
      #print(nombre_archivo.split("_", -1)[-1])
      #print(tablas)
      tablas = tablas.T.iloc[2:,[1, 5]].reset_index(drop=True).rename(columns={1:'Periodo_trimestral',5:'Consumo_de_hogares_e_IPSFL'})
      tablas['Region'] = nombre_archivo.split("_", -1)[-1]
      tablas = tablas[['Periodo_trimestral','Consumo_de_hogares_e_IPSFL','Region']]
      df_consumo = pd.concat([df_consumo,tablas],axis=0)  # Concatena cada DataFrame por región

In [6]:
df_consumo['Periodo_trimestral'] = pd.to_datetime(df_consumo['Periodo_trimestral'])
df_consumo

,Periodo_trimestral,Consumo_de_hogares_e_IPSFL,Region
0,2013-03-01,393.516067,Tarapaca
1,2013-06-01,396.035933,Tarapaca
2,2013-09-01,414.969278,Tarapaca
3,2013-12-01,447.75219,Tarapaca
4,2014-03-01,402.835411,Tarapaca
...,...,...,...
41,2023-06-01,16383.297847,Metropolitana
42,2023-09-01,16034.061789,Metropolitana
43,2023-12-01,17149.969954,Metropolitana
44,2024-03-01,15553.20325,Metropolitana



Tabla trimestral.

*   Solo se tiene información desde el 2013 hasta el segundo trimestre de el presente año.
*   Información completa para cada región



In [7]:
df_consumo.to_parquet('/content/drive/MyDrive/CD-FINAL/datos de economia/Datos_ordenados_1/df_consumo_trimestral.parquet', engine='pyarrow')



---


### TABLA2: Tenencia y saldos en cuentas corrientes y vista (Indicadores financieros)

Número de cuentas corrientes de personas naturales en moneda nacional
- **Descripción:**  
Es la cantidad total de cuentas corrientes pertenecientes a personas naturales (individuos, no empresas) que están denominadas en pesos chilenos.
Indica cuántas personas tienen cuentas corrientes activas en moneda nacional.

Saldo promedio de cuentas corrientes de personas naturales en moneda nacional (en pesos)
- **Descripción:**  
Representa el saldo promedio disponible en las cuentas corrientes de personas naturales denominadas en pesos chilenos.
Refleja el comportamiento financiero general de los usuarios en términos de cuánto dinero suelen mantener en sus cuentas.

Número de cuentas corrientes de personas naturales en moneda extranjera
- **Descripción:**  
Es la cantidad total de cuentas corrientes de personas naturales denominadas en una moneda extranjera (por ejemplo, dólares estadounidenses).
Muestra cuántas personas han optado por mantener cuentas en monedas extranjeras, lo que puede estar relacionado con actividades internacionales o protección contra devaluaciones del peso chileno.

Saldo promedio de cuentas corrientes de personas naturales en moneda extranjera (en pesos)
- **Descripción:**  
Representa el saldo promedio en cuentas corrientes denominadas en monedas extranjeras, convertido a pesos chilenos.
Da una idea de cuánto dinero manejan las personas en cuentas extranjeras, lo que puede ser útil para entender tendencias de dolarización o ahorro en otras monedas.

Saldo de cuentas a la vista por región (en millones de pesos)
- **Descripción:**  
Representa el monto total disponible en cuentas a la vista (similares a cuentas corrientes, pero con menor complejidad) expresado en millones de pesos chilenos.
Es una métrica de liquidez en la región, mostrando cuánto dinero está disponible de manera inmediata en estas cuentas.

In [8]:
archivos_csv = glob.glob(os.path.join(directorio, "*.xlsx"))

# Almancenaré en un diccionario los dataframe
df_Ind_finan = pd.DataFrame()

# Leer cada archivo CSV y almacenarlo en el diccionario
for i in archivos_csv:
    nombre_archivo = os.path.basename(i).split('.')[0]  # Nombre del archivo sin la extensión
    if 'Ind_finan' in nombre_archivo:
      tablas = pd.read_excel(i, engine='openpyxl')
      tablas.columns = tablas.iloc[1]
      tablas = tablas.iloc[2:].reset_index(drop=True)
      tablas = tablas.rename(columns={tablas.columns[-1]:'5.Saldo de cuentas a la vista (en millones de pesos)'})
      tablas['Region'] =  nombre_archivo.split("_", -1)[-1]
      df_Ind_finan = pd.concat([df_Ind_finan,tablas],axis=0)
      #df_consumo = pd.concat([df_consumo,tablas],axis=0)  # Guarda el DataFrame en el diccionario con el nombre del archivo como clave

In [9]:
df_Ind_finan['Periodo'] = pd.to_datetime(df_Ind_finan['Periodo'])
df_Ind_finan['Año_Trimestre'] = df_Ind_finan['Periodo'].dt.to_period('Q')
df_Ind_finan['Periodo_trimestral'] = df_Ind_finan['Año_Trimestre'].apply(lambda x: (x.start_time + pd.DateOffset(months=2)))
df_Ind_finan.drop(columns={'Periodo','Año_Trimestre'},inplace=True)
df_Ind_finan_trimes = df_Ind_finan.groupby(['Region','Periodo_trimestral']).mean().reset_index()
df_Ind_finan_trimes

1,Region,Periodo_trimestral,1.Número de cuentas corrientes de personas naturales en moneda nacional,2.Saldo promedio de cuentas corrientes de personas naturales en moneda nacional (en pesos),3.Número de cuentas corrientes de personas naturales en moneda extranjera,4.Saldo promedio de cuentas corrientes de personas naturales en moneda extranjera (en pesos),5.Saldo de cuentas a la vista (en millones de pesos)
0,Antofagasta,2013-03-01,96238.333333,2324128.063219,320.0,8372920.426099,53273.287224
1,Antofagasta,2013-06-01,98320.666667,2183286.573678,324.333333,8524981.876501,54060.851445
2,Antofagasta,2013-09-01,100339.0,2097439.229556,330.666667,7060843.187003,52443.110412
3,Antofagasta,2013-12-01,102117.666667,2046252.577891,333.666667,7321857.956338,56020.125436
4,Antofagasta,2014-03-01,103377.0,2133526.007748,334.333333,9050310.192042,56750.909477
...,...,...,...,...,...,...,...
724,Valparaiso,2023-09-01,400536.666667,2222853.341999,30433.333333,1966408.822265,551433.063098
725,Valparaiso,2023-12-01,404575.333333,2190261.081638,32552.666667,1787956.044287,551114.390586
726,Valparaiso,2024-03-01,414492.333333,2251444.034605,36026.333333,1540624.937593,573778.864104
727,Valparaiso,2024-06-01,422535.666667,2248382.459183,39539.0,1416143.537496,580367.59605


Tabla mensual convertida a trimestral.

*   Se tiene información desde el 2013 hasta el tercer trimestre de el presente año.
*   Información completa para cada región, excepto para la region de Ñuble ya que solo tiene info desde el 2018


In [10]:
df_Ind_finan_trimes.to_parquet('/content/drive/MyDrive/CD-FINAL/datos de economia/Datos_ordenados_1/df_Ind_finan_trimes.parquet', engine='pyarrow')



---


### TABLA3: Fuerza de Trabajo, Ocupados y Tasa de Desocupación  


Fuerza de trabajo (Miles de personas)  
- **Descripción:**  
  Representa el total de personas económicamente activas, es decir, aquellas que están trabajando o buscan empleo activamente.  

Ocupados (Miles de personas)  
- **Descripción:**  
  Número de personas dentro de la fuerza laboral que están empleadas (que tienen un trabajo).  

Tasa de desocupación (%)  
- **Descripción:**  
  Porcentaje de personas en la fuerza laboral que están desocupadas (buscando empleo activamente pero sin trabajo).  




In [11]:
archivos_csv = glob.glob(os.path.join(directorio, "*.xlsx"))

# Almancenaré en un diccionario los dataframe
df_Ind_des = pd.DataFrame()

# Leer cada archivo CSV y almacenarlo en el diccionario
for i in archivos_csv:
    nombre_archivo = os.path.basename(i).split('.')[0]  # Nombre del archivo sin la extensión
    if 'Ind_des' in nombre_archivo:
      tablas = pd.read_excel(i, engine='openpyxl')
      tablas.columns = tablas.iloc[1]
      tablas = tablas.iloc[2:].reset_index(drop=True)
      tablas['Region'] = nombre_archivo.split('_',-1)[-1]
      df_Ind_des = pd.concat([df_Ind_des,tablas],axis=0)  # Guarda el DataFrame en el diccionario con el nombre del archivo como clave

In [12]:
df_Ind_des

1,Periodo,1.Fuerza de trabajo (Miles de personas),2.Ocupados (Miles de personas),3.Tasa de desocupación (%),Region
0,2010-03-01 00:00:00,128.839356,117.058364,9.143939,Atacama
1,2010-04-01 00:00:00,128.544415,118.829328,7.557767,Atacama
2,2010-05-01 00:00:00,131.889451,120.441679,8.679824,Atacama
3,2010-06-01 00:00:00,131.261722,120.475363,8.217445,Atacama
4,2010-07-01 00:00:00,130.986821,120.494326,8.010345,Atacama
...,...,...,...,...,...
170,2024-05-01 00:00:00,240.244768,215.664888,10.231182,Nuble
171,2024-06-01 00:00:00,236.682592,211.28471,10.730777,Nuble
172,2024-07-01 00:00:00,230.596069,205.727593,10.784432,Nuble
173,2024-08-01 00:00:00,233.470493,206.985864,11.343887,Nuble


In [13]:
df_Ind_des['Periodo'] = pd.to_datetime(df_Ind_des['Periodo'])
df_Ind_des = df_Ind_des[df_Ind_des['Periodo']>'2012-12-01']
df_Ind_des['Año_Trimestre'] = df_Ind_des['Periodo'].dt.to_period('Q')
df_Ind_des['Periodo_trimestral'] = df_Ind_des['Año_Trimestre'].apply(lambda x: (x.start_time + pd.DateOffset(months=2)))
df_Ind_des.drop(columns={'Periodo','Año_Trimestre'},inplace=True)
df_Ind_des_trimes = df_Ind_des.groupby(['Region','Periodo_trimestral']).mean().reset_index()


df_Ind_des_trimes

<ipython-input-13-ea90425743ae>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Ind_des['Año_Trimestre'] = df_Ind_des['Periodo'].dt.to_period('Q')
<ipython-input-13-ea90425743ae>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Ind_des['Periodo_trimestral'] = df_Ind_des['Año_Trimestre'].apply(lambda x: (x.start_time + pd.DateOffset(months=2)))
<ipython-input-13-ea90425743ae>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

1,Region,Periodo_trimestral,1.Fuerza de trabajo (Miles de personas),2.Ocupados (Miles de personas),3.Tasa de desocupación (%)
0,Antofagasta,2013-03-01,269.113297,253.510501,5.797311
1,Antofagasta,2013-06-01,277.229068,262.261608,5.399044
2,Antofagasta,2013-09-01,273.138637,258.056848,5.521059
3,Antofagasta,2013-12-01,280.224621,262.012615,6.500208
4,Antofagasta,2014-03-01,278.252303,257.621397,7.413592
...,...,...,...,...,...
747,Valparaiso,2023-09-01,994.064092,903.022493,9.157725
748,Valparaiso,2023-12-01,993.825406,910.162555,8.418221
749,Valparaiso,2024-03-01,1001.435912,911.514531,8.979508
750,Valparaiso,2024-06-01,992.174294,911.392222,8.142903


Tabla mensual convertida a trimestral.

*   Se tiene información desde el 2013 hasta el tercer trimestre de el presente año.
*   Información completa para cada región.

In [14]:
df_Ind_des_trimes.to_parquet('/content/drive/MyDrive/CD-FINAL/datos de economia/Datos_ordenados_1/df_Ind_des_trimes.parquet', engine='pyarrow')

## Merge de las tablas

In [15]:
path = '/content/drive/MyDrive/CD-FINAL/datos de economia/Datos_ordenados_1/'
#path = 'Datos_ordenados_1/'

In [16]:
df_consumo_trimes = pd.read_parquet(f'{path}df_consumo_trimestral.parquet', engine='pyarrow')
df_Ind_finan_trimes = pd.read_parquet(f'{path}df_Ind_finan_trimes.parquet', engine='pyarrow')
df_Ind_des_trimes = pd.read_parquet(f'{path}df_Ind_des_trimes.parquet', engine='pyarrow')
df_pib_reg_trimes = pd.read_parquet(f'{path}pib_regional.parquet', engine='pyarrow')

In [17]:
print('df_consumo_trimes:',df_consumo_trimes.shape)
print('df_Ind_finan_trimes:',df_Ind_finan_trimes.shape)
print('df_Ind_des_trimes:',df_Ind_des_trimes.shape)
print('df_pib_reg_trimes:',df_pib_reg_trimes.shape)

df_pib_reg_trimes.rename(columns={"fecha": "Periodo_trimestral", 'region': 'Region', "PIB": "pib regional"}, inplace=True)

df_consumo_trimes: (736, 3)
df_Ind_finan_trimes: (729, 7)
df_Ind_des_trimes: (752, 5)
df_pib_reg_trimes: (736, 3)


In [45]:
for df in [df_consumo_trimes, df_Ind_des_trimes, df_Ind_finan_trimes, df_pib_reg_trimes]:
    df["Cod_Region"] = df['Region'].apply(lambda x: codigo_regiones.get(x, x))
df['Cod_Region'] = df['Cod_Region'].astype(str)
df['Cod_Region'] = df['Cod_Region'].replace('Aysén',11)
df['Cod_Region'] = df['Cod_Region'].astype(int)

In [56]:
df_economia_final = df_consumo_trimes.drop(columns={'Region'}).merge(df_Ind_finan_trimes, how = 'left', on = ['Periodo_trimestral', "Cod_Region"])
df_economia_final = df_economia_final.merge(df_Ind_des_trimes, how = 'left', on = ['Periodo_trimestral', "Cod_Region"])
df_economia_final = df_economia_final.merge(df_pib_reg_trimes, how = 'left', on=['Periodo_trimestral', "Cod_Region"])

df_economia_final["Cod_Region"] = df_economia_final["Cod_Region"].astype(int)

In [57]:
df_economia_final.info()
df_economia_final

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 15 columns):
 #   Column                                                                                        Non-Null Count  Dtype         
---  ------                                                                                        --------------  -----         
 0   Periodo_trimestral                                                                            736 non-null    datetime64[ns]
 1   Consumo_de_hogares_e_IPSFL                                                                    736 non-null    float64       
 2   Cod_Region                                                                                    736 non-null    int64         
 3   Region_x                                                                                      713 non-null    object        
 4   1.Número de cuentas corrientes de personas naturales en moneda nacional                       713 non-null    

,Periodo_trimestral,Consumo_de_hogares_e_IPSFL,Cod_Region,Region_x,1.Número de cuentas corrientes de personas naturales en moneda nacional,2.Saldo promedio de cuentas corrientes de personas naturales en moneda nacional (en pesos),3.Número de cuentas corrientes de personas naturales en moneda extranjera,4.Saldo promedio de cuentas corrientes de personas naturales en moneda extranjera (en pesos),5.Saldo de cuentas a la vista (en millones de pesos),Region_y,1.Fuerza de trabajo (Miles de personas),2.Ocupados (Miles de personas),3.Tasa de desocupación (%),pib regional,Region
0,2013-03-01,393.516067,1,Tarapaca,4.139133e+04,1.799732e+06,666.333333,1.092781e+07,1.928896e+04,Tarapaca,142.670496,136.036022,4.647879,811.931803,Tarapaca
1,2013-06-01,396.035933,1,Tarapaca,4.204000e+04,1.785954e+06,664.666667,1.101341e+07,2.021345e+04,Tarapaca,151.111341,140.688553,6.885483,926.814620,Tarapaca
2,2013-09-01,414.969278,1,Tarapaca,4.273100e+04,1.853857e+06,669.000000,1.116178e+07,2.027268e+04,Tarapaca,154.507437,145.272515,5.976382,1044.963002,Tarapaca
3,2013-12-01,447.752190,1,Tarapaca,4.340833e+04,1.832446e+06,667.666667,1.289925e+07,2.150547e+04,Tarapaca,155.474256,145.933936,6.137231,1135.992471,Tarapaca
4,2014-03-01,402.835411,1,Tarapaca,4.393533e+04,1.909806e+06,668.000000,1.379185e+07,2.182164e+04,Tarapaca,158.197040,147.823280,6.557147,977.620431,Tarapaca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,2023-06-01,16383.297847,13,Metropolitana,6.065315e+06,1.209344e+06,429379.333333,2.209989e+06,3.777679e+06,Metropolitana,4443.822474,4033.025196,9.244615,21281.075679,Metropolitana
732,2023-09-01,16034.061789,13,Metropolitana,6.517317e+06,1.106488e+06,480266.666667,1.787989e+06,3.662006e+06,Metropolitana,4469.990997,4044.122061,9.527413,20673.921435,Metropolitana
733,2023-12-01,17149.969954,13,Metropolitana,7.034871e+06,1.037563e+06,533406.000000,1.535191e+06,3.735128e+06,Metropolitana,4497.930053,4065.239972,9.620284,22032.121888,Metropolitana
734,2024-03-01,15553.203250,13,Metropolitana,7.459227e+06,1.034034e+06,581706.000000,1.379730e+06,3.833088e+06,Metropolitana,4525.393446,4102.689078,9.340321,21124.552771,Metropolitana


- Se hizo un merge tomando como referencia el tamaño de la tabla de consumo ya que solo cuenta con información hasta el segundo trimestre de este periodo.
- Se observa que hay datos faltantes, es debido que la región Ñuble no cuenta con esa información en la página del Banco Central.

In [58]:
columnas = {
    'Periodo_trimestral': 'Periodo_trimestral',
    'Consumo_de_hogares_e_IPSFL': 'Consumo_Hogares',
    'Region': 'Region',
    'Cod_Region': 'CodRegion',
    '1.Número de cuentas corrientes de personas naturales en moneda nacional': 'Ctas_Nacional',
    '2.Saldo promedio de cuentas corrientes de personas naturales en moneda nacional (en pesos)': 'Saldo_Nacional',
    '3.Número de cuentas corrientes de personas naturales en moneda extranjera': 'Ctas_Extranjera',
    '4.Saldo promedio de cuentas corrientes de personas naturales en moneda extranjera (en pesos)': 'Saldo_Extranjero',
    '5.Saldo de cuentas a la vista (en millones de pesos)': 'Saldo_Vista',
    '1.Fuerza de trabajo (Miles de personas)': 'Fuerza_Trabajo',
    '2.Ocupados (Miles de personas)': 'Ocupados',
    '3.Tasa de desocupación (%)': 'Tasa_Desocupacion',
    'pib regional': 'PIB_Regional'
}

# Aplicar el cambio de nombre
df_economia_final.rename(columns=columnas, inplace=True)

In [59]:
df_economia_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Periodo_trimestral  736 non-null    datetime64[ns]
 1   Consumo_Hogares     736 non-null    float64       
 2   CodRegion           736 non-null    int64         
 3   Region_x            713 non-null    object        
 4   Ctas_Nacional       713 non-null    float64       
 5   Saldo_Nacional      713 non-null    float64       
 6   Ctas_Extranjera     713 non-null    float64       
 7   Saldo_Extranjero    713 non-null    float64       
 8   Saldo_Vista         713 non-null    float64       
 9   Region_y            736 non-null    object        
 10  Fuerza_Trabajo      736 non-null    float64       
 11  Ocupados            736 non-null    float64       
 12  Tasa_Desocupacion   736 non-null    float64       
 13  PIB_Regional        736 non-null    float64       

In [60]:
df_economia_final.to_parquet('/content/drive/MyDrive/CD-FINAL/datos de economia/df_economia_final.parquet', engine='pyarrow')
#df_economia_final.to_parquet('df_economia_final.parquet', engine='pyarrow')